In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.kompas.com/"
articles_data = []
max_articles = 1000  # Jumlah maksimal artikel yang ingin diambil

# Fungsi untuk mengambil konten, penulis, dan gambar dari detail artikel
def get_article_content(article_url):
    try:
        page = requests.get(article_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Ambil isi artikel
        content_paragraphs = soup.find('div', class_='read__content').find_all('p')
        content = "\n".join([p.text.strip() for p in content_paragraphs])

        # Ambil penulis dari class 'credit-title-nameEditor'
        penulis_tags = soup.find_all('div', class_='credit-title-nameEditor')
        penulis = ', '.join([tag.text.strip() for tag in penulis_tags]) if penulis_tags else 'Tidak diketahui'

        # Ambil gambar utama dari div.photo__wrap > img
        gambar_url = ''
        photo_wrap = soup.find('div', class_='photo__wrap')
        if photo_wrap:
            img = photo_wrap.find('img')
            if img and img.has_attr('src'):
                gambar_url = img['src']
        
        # Ambil kategori dari breadcrumb
        # Ambil kategori utama dari breadcrumb
        kategori = 'Tidak diketahui'
        breadcrumb_items = soup.select('ul.breadcrumb__wrap li.breadcrumb__item')
        if len(breadcrumb_items) > 1:
            kategori = breadcrumb_items[1].text.strip()


        return content, penulis, gambar_url, kategori

    except Exception as e:
        print(f"Gagal mengambil detail dari {article_url}: {e}")
        return "", "Tidak diketahui", ""

# Loop untuk mengambil artikel dari halaman indeks
for page_num in range(1, 300):
    if len(articles_data) >= max_articles:
        break

    url = f"https://indeks.kompas.com/?site=bola&page={page_num}"
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        articles = soup.find_all('div', class_='articleItem')

        for article in articles:
            if len(articles_data) >= max_articles:
                break

            article_url = article.find('a', class_='article-link')['href']
            title = article.find('h2', class_='articleTitle').text.strip()
            date = article.find('div', class_='articlePost-date').text.strip()

            # Ambil detail dari halaman artikel
            content, penulis, gambar_url, kategori = get_article_content(article_url)

            # Simpan data artikel
            articles_data.append({
                'judul': title,
                'kategori': kategori,
                'tanggal': date,
                'konten': content,
                'penulis': penulis,
                'gambar': gambar_url,
                'link': article_url,
                'sumber': BASE_URL

            })

            # Tampilkan progress
            print(f"[{len(articles_data)}] Berhasil ambil: {title}")

    except Exception as e:
        print(f"Error pada halaman {page_num}: {e}")
        continue

# Simpan ke DataFrame
df = pd.DataFrame(articles_data)

# Simpan ke CSV
df.to_csv('hasil/kompas_bola_articles.csv', index=False)

# Simpan ke JSON
df.to_json('hasil/kompas_bola_articles.json', orient='records', lines=True, force_ascii=False)

# Tampilkan ringkasan
print(f"\n Total artikel berhasil diambil: {len(df)}")


[1] Berhasil ambil: Man United ke Malaysia Pekan Ini, Laga Lawan ASEAN All Stars Sulit
[2] Berhasil ambil: Kylian Mbappe Raih Sepatu Emas Eropa 2025 Usai Mo Salah Gagal Hattrick
[3] Berhasil ambil: Klasemen Akhir Liga Inggris 2024-2025: Liverpool Juara, Chelsea dan Newcastle Lolos Liga Champions
[4] Berhasil ambil: Skor Athletic Bilbao Vs Barcelona 0-3: Rekor 101 Gol Lewandowski
[5] Berhasil ambil: Klasemen MotoGP 2025 Usai Drama GP Inggris: Marc Marquez Masih Teratas
[6] Berhasil ambil: Hasil dan Klasemen Liga Italia: Juventus Kirim Tim Jay Idzes ke Serie B
[7] Berhasil ambil: Skor Manchester United Vs Aston Villa 2-0: MU Pupuskan Asa The Villans ke Liga Champions
[8] Berhasil ambil: Hasil Athletic Bilbao Vs Barcelona 0-3: Lewandowski dan Barca Tembus 100 Gol
[9] Berhasil ambil: Hasil Venezia Vs Juventus 2-3: Bianconeri ke Liga Champions, Tim Jay Idzes Degradasi
[10] Berhasil ambil: Hasil Drawing Piala Dunia U17 2025: Indonesia Satu Grup dengan Brasil
[11] Berhasil ambil: Hasil Undian

: 